In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier as RFC

In [2]:
tweet_df_location = pd.read_csv('../data/graphAnalysis/clean_climateTwitterData.csv')
tweet_df_location = tweet_df_location[tweet_df_location['text'].notna()]

In [3]:
import re
import string
from nltk.tokenize import TweetTokenizer

# This step already done in Sentiment Analyzer
# def replace_urls(in_string, replacement=None):
#     # """Replace URLs in strings. See also: ``bit.ly/PyURLre``

#     # Args:
#     #     in_string (str): string to filter
#     #     replacement (str or None): replacment text. defaults to '<-URL->'

#     # Returns:
#     #     str
#     # """
#     replacement = '<-URL->' if replacement is None else replacement
#     pattern = re.compile('(https?://)?(\w*[.]\w+)+([/?=&]+\w+)*')
#     return re.sub(pattern, replacement, in_string)


def tokenize_only(in_string):
    """
    Convert `in_string` of text to a list of tokens using NLTK's TweetTokenizer
    """
    # reasonable, but adjustable tokenizer settings
    tokenizer = TweetTokenizer(preserve_case=False,
                               reduce_len=True,
                               strip_handles=False)
    tokens = tokenizer.tokenize(in_string)
    return tokens



In [4]:
# This step already done in Sentiment Analyzer
# # remove urls and retweets and entities from the text
# tweet_df_location['text_clean'] = tweet_df_location['text'].apply(lambda row:replace_urls(row))

# #remove punctuations
# RE_PUNCTUATION = '|'.join([re.escape(x) for x in string.punctuation])  
# tweet_df_location['text_clean'] = tweet_df_location['text_clean'].str.replace(RE_PUNCTUATION, "")

In [4]:
from nltk.corpus import stopwords

# List of stopwords
stop_words= stopwords.words('english') #import stopwords from NLTK package
readInStopwords = pd.read_csv("pre_process/twitterStopWords.csv", encoding='ISO-8859-1') # import stopwords from CSV file as pandas data frame
readInStopwords = readInStopwords.wordList.tolist() # convert pandas data frame to a list
readInStopwords.append('http')
readInStopwords.append('https')

search_terms = ['#climateStrike','#climatestrike','#climatechange','#GreenNewDeal','#climatecrisis','#climateAction','#FridaysForFuture',
            '#environment','#globalwarming','#GlobalWarming','#ActOnClimate','#sustainability','#savetheplanet',
        '#bushfiresAustralia','#bushfires']
readInStopwords.extend(search_terms)
stop_list = stop_words + readInStopwords # combine two lists i.e. NLTK stop words and CSV stopwords
stop_list = list(set(stop_list)) # strore only unique values 
    
print(stop_list)

["you'll", 'the', 'o', 'just', "where's", 'wasn', "he'd", ' its', ' says', 'y', 'up', ' wants', 'then', ' he', 'wouldn', 'our', ' them', 'after', ' there', 'should', 'shouldn', 'i', "didn't", "hasn't", 'll', 'so', " should've", ' but', ' else', " couldn't", ' in', 'its', "let's", ' other', ' said', " i've", "it's", 'on', "he's", "you've", 'further', '#GreenNewDeal', '#sustainability', 'before', ' also', "why's", 'could', ' his', " why's", " might've", 'too', ' neither', ' nor', " i'll", ' did', " don't", 'between', 'once', ' she', '#globalwarming', 'their', 'few', ' ever', "'tis", '#environment', "mightn't", 'hasn', " they'd", ' of', ' should', ' must', ' us', 'they', ' likely', 'myself', 'can', " you'd", 'by', 'didn', "we're", ' were', ' had', ' every', " he's", 'other', 'which', " what's", 'is', 'from', " mustn't", ' and', ' an', ' with', " we'd", 'aren', ' too', ' no', 'http', " can't", 'most', 'her', ' her', ' or', 'my', 'under', ' hers', " it's", 'over', '#climateStrike', "i've", 

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X = tweet_df_location['text_clean']
y = tweet_df_location['search_hashtags']


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf_vectorizer = TfidfVectorizer(max_df=0.9, min_df=0.00, stop_words=stop_list, tokenizer=tokenize_only) # Use tf (raw term count) features for LDA.
X_Vect = tf_vectorizer.fit_transform(X)

c:\users\tasne\envs\newproject\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['#actonclimate', '#bushfiresaustralia', '#climateaction', '#fridaysforfuture', '#greennewdeal', "'", 'able', 'across', "ain't", 'almost', 'also', 'among', "can't", 'cannot', "could've", 'dear', 'either', 'else', 'ever', 'every', 'get', 'got', "how'd", "how'll", 'however', 'least', 'let', 'like', 'likely', 'may', 'might', "might've", 'must', "must've", 'neither', 'often', 'rather', 'said', 'say', 'says', 'since', 'tis', 'twas', 'us', 'wants', "what'd", "when'd", "when'll", "where'd", "where'll", "who'd", "who'll", "why'd", "why'll", "would've", 'yet'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_Vect, y, test_size=0.30, random_state=42, stratify=y)

In [9]:
X_train.shape

(50683, 87818)

In [10]:
y_train.shape

(50683,)

In [11]:
X_test.shape

(21722, 87818)

In [12]:
y_test.shape

(21722,)

In [13]:
rfc = RFC()

In [14]:
rfc.fit(X_train, y_train)  

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
predictions = rfc.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
print(confusion_matrix(y_test,predictions))

In [ ]:
print(classification_report(y_test,predictions))

In [ ]:
from sklearn import metrics

In [ ]:
metrics.accuracy_score(y_test,predictions)

0.7970720928091336 -- Accuracy

In [ ]:
strVal = ["People are getting aware of the surrounding ","things are super bad"]
Value = text_Vect.transform(strVal)
Value.shape

In [ ]:
rfc.predict(Value)